In [93]:
from compChem.Hartree_Fock import Molecule
from scipy.linalg import eigh, sqrtm
import numpy as np
import psi4

In [94]:
def basischanger(c, f):
        """
        allows a change in basis
        
        input:
        c: transformation matrix
        f: operator to be transformed
        """
        return np.einsum("ij,jm,mn->in", c.T, f, c, optimize=True)

In [4]:
class ConstrainedMolecule:
    def __init__(self, geom_file):
        """initiation method, will take in a geometry"""
        #this parameter contains all parameters and methods from the Molecule class
        self.id = Molecule(geom_file)
        self.id.setGuess()
        
        #getting some old and some new matrices
        self.gamma_alpha = self.id.getDensityMatrix("alpha")
        self.gamma_beta = self.id.getDensityMatrix("beta")
        self.pMatrix = (self.gamma_alpha + self.gamma_beta)/2
        self.transform = eigh(self.pMatrix)[1]
        
    
    def fock_Alternator(self, spin):
        "alters a fock operator to follow CUHF"
        assert spin == "alpha" or spin == "beta", f"{spin}: no valid spin"
        f = self.id.displayFockMatrix(spin)
        f_trans = basischanger(self.transform, f)
        coulomb_a = np.einsum("nopq,pq->no", self.id.displayElectronRepulsion(), self.gamma_beta, optimize=True)
        exchange = np.einsum("npoq,pq->no", self.id.displayElectronRepulsion(), self.gamma_beta, optimize=True)
        closed_shell_type = 0.5*(self.id.displayHamiltonian() + 2*coulomb_a - exchange)
        closed_shell_trans = basischanger(self.transform, closed_shell_type)
        
        # amount of paired electrons
        cc = self.id.beta
        
        # amount of unpaored electrons
        oo = self.id.alpha - self.id.beta
        
        # amount of unoccupied orbitals
        
        # alter first block
        f_trans[:cc + oo, cc + oo:] = closed_shell_trans[:cc + oo, cc + oo:]
        
        # alter block in transposed position
        f_trans[cc + oo:, :cc + oo] = closed_shell_trans[cc + oo:, :cc + oo]
        
        
        return f_trans
        
    def iteratinator(self):
        self.id.setGuess(self.fock_Alternator("alpha"), "alpha")
        self.id.setGuess(self.fock_Alternator("beta"), "beta")
        return self.id.iterator(iteration=1000)
    
    
    def updatePMatrix(self):
        """updates pMatrix to current self.id densitymatrices"""
        self.pMatrix = (self.id.getDensityMatrix("alpha") + self.id.getDensityMatrix("beta"))

In [5]:
psi4.set_options({"BASIS": "cc-pvdz"})
h1 = ConstrainedMolecule("""
H 0 0 0
H 0 0.86602540378 0.5
H 0 0 1
units angstrom""")
h1.id.setConvergence(1e-10)
h1.iteratinator()

/user/gent/440/vsc44013/.local/lib/python3.7/site-packages/compChem/Hartree_Fock.py:56: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if self.guessMatrix_a == "empty" and self.guessMatrix_b == "empty":
/user/gent/440/vsc44013/.local/lib/python3.7/site-packages/compChem/Hartree_Fock.py:210: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  assert self.guessMatrix_a != "empty" and self.guessMatrix_b != "empty", "make a guess first"


iteration: 0, E_tot:  2.91464457, E_elek:  1.32711294, deltaE:  1.32711294, rmsD:  32.00852210
iteration: 1, E_tot: -1.44414734, E_elek: -3.03167897, deltaE: -4.35879191, rmsD:  2.97905133
iteration: 2, E_tot: -1.49856885, E_elek: -3.08610048, deltaE: -0.05442151, rmsD:  0.86650854
iteration: 3, E_tot: -1.50432654, E_elek: -3.09185817, deltaE: -0.00575769, rmsD:  0.33152760
iteration: 4, E_tot: -1.50547028, E_elek: -3.09300191, deltaE: -0.00114374, rmsD:  0.19512637
iteration: 5, E_tot: -1.50582372, E_elek: -3.09335535, deltaE: -0.00035344, rmsD:  0.14091222
iteration: 6, E_tot: -1.50596358, E_elek: -3.09349521, deltaE: -0.00013986, rmsD:  0.11058976
iteration: 7, E_tot: -1.50602771, E_elek: -3.09355935, deltaE: -0.00006413, rmsD:  0.09210649
iteration: 8, E_tot: -1.50606117, E_elek: -3.09359280, deltaE: -0.00003345, rmsD:  0.08116712
iteration: 9, E_tot: -1.50608136, E_elek: -3.09361299, deltaE: -0.00002019, rmsD:  0.07386870
iteration: 10, E_tot: -1.50609555, E_elek: -3.09362718, del

iteration: 94, E_tot: -1.50627341, E_elek: -3.09380504, deltaE: -0.00000006, rmsD:  0.00457823
iteration: 95, E_tot: -1.50627347, E_elek: -3.09380510, deltaE: -0.00000006, rmsD:  0.00443207
iteration: 96, E_tot: -1.50627352, E_elek: -3.09380515, deltaE: -0.00000005, rmsD:  0.00429059
iteration: 97, E_tot: -1.50627357, E_elek: -3.09380520, deltaE: -0.00000005, rmsD:  0.00415363
iteration: 98, E_tot: -1.50627362, E_elek: -3.09380525, deltaE: -0.00000005, rmsD:  0.00402107
iteration: 99, E_tot: -1.50627366, E_elek: -3.09380529, deltaE: -0.00000004, rmsD:  0.00389274
iteration: 100, E_tot: -1.50627370, E_elek: -3.09380533, deltaE: -0.00000004, rmsD:  0.00376853
iteration: 101, E_tot: -1.50627374, E_elek: -3.09380537, deltaE: -0.00000004, rmsD:  0.00364829
iteration: 102, E_tot: -1.50627378, E_elek: -3.09380541, deltaE: -0.00000004, rmsD:  0.00353189
iteration: 103, E_tot: -1.50627381, E_elek: -3.09380544, deltaE: -0.00000003, rmsD:  0.00341922
iteration: 104, E_tot: -1.50627384, E_elek: -3

iteration: 190, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00020520
iteration: 191, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00019868
iteration: 192, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00019237
iteration: 193, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00018626
iteration: 194, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00018034
iteration: 195, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00017461
iteration: 196, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00016906
iteration: 197, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00016369
iteration: 198, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00015848
iteration: 199, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00015345
iteration: 200, E_tot: -1.50627432, E_el

iteration: 286, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00000925
iteration: 287, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00000895
iteration: 288, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00000867
iteration: 289, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00000839
iteration: 290, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00000813
iteration: 291, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00000787
iteration: 292, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00000762
iteration: 293, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00000738
iteration: 294, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00000714
iteration: 295, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00000692
iteration: 296, E_tot: -1.50627432, E_el

iteration: 382, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00000042
iteration: 383, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00000040
iteration: 384, E_tot: -1.50627432, E_elek: -3.09380595, deltaE:  0.00000000, rmsD:  0.00000039
iteration: 385, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00000038
iteration: 386, E_tot: -1.50627432, E_elek: -3.09380595, deltaE:  0.00000000, rmsD:  0.00000037
iteration: 387, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00000035
iteration: 388, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00000034
iteration: 389, E_tot: -1.50627432, E_elek: -3.09380595, deltaE:  0.00000000, rmsD:  0.00000033
iteration: 390, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00000032
iteration: 391, E_tot: -1.50627432, E_elek: -3.09380595, deltaE:  0.00000000, rmsD:  0.00000031
iteration: 392, E_tot: -1.50627432, E_el

iteration: 478, E_tot: -1.50627432, E_elek: -3.09380595, deltaE:  0.00000000, rmsD:  0.00000002
iteration: 479, E_tot: -1.50627432, E_elek: -3.09380595, deltaE:  0.00000000, rmsD:  0.00000002
iteration: 480, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00000002
iteration: 481, E_tot: -1.50627432, E_elek: -3.09380595, deltaE:  0.00000000, rmsD:  0.00000002
iteration: 482, E_tot: -1.50627432, E_elek: -3.09380595, deltaE:  0.00000000, rmsD:  0.00000002
iteration: 483, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00000002
iteration: 484, E_tot: -1.50627432, E_elek: -3.09380595, deltaE:  0.00000000, rmsD:  0.00000002
iteration: 485, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00000001
iteration: 486, E_tot: -1.50627432, E_elek: -3.09380595, deltaE:  0.00000000, rmsD:  0.00000001
iteration: 487, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00000001
iteration: 488, E_tot: -1.50627432, E_el

iteration: 574, E_tot: -1.50627432, E_elek: -3.09380595, deltaE:  0.00000000, rmsD:  0.00000000
iteration: 575, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00000000
iteration: 576, E_tot: -1.50627432, E_elek: -3.09380595, deltaE:  0.00000000, rmsD:  0.00000000
iteration: 577, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00000000
iteration: 578, E_tot: -1.50627432, E_elek: -3.09380595, deltaE:  0.00000000, rmsD:  0.00000000
iteration: 579, E_tot: -1.50627432, E_elek: -3.09380595, deltaE:  0.00000000, rmsD:  0.00000000
iteration: 580, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00000000
iteration: 581, E_tot: -1.50627432, E_elek: -3.09380595, deltaE:  0.00000000, rmsD:  0.00000000
iteration: 582, E_tot: -1.50627432, E_elek: -3.09380595, deltaE:  0.00000000, rmsD:  0.00000000
iteration: 583, E_tot: -1.50627432, E_elek: -3.09380595, deltaE:  0.00000000, rmsD:  0.00000000
iteration: 584, E_tot: -1.50627432, E_el

-1.506274320299386

In [64]:
psi4.set_options({"BASIS": "cc-pvdz"})
h_test = ConstrainedMolecule("""
H 0 0 0
H 0 0.86602540378 0.5
H 0 0 1
units angstrom""")


In [65]:


c = eigh(f_a)

s = h_test.id.overlap
p = h_test.pMatrix
s_eigh = eigh(s)
s_diag = np.diag(s_eigh[0]**(-1/2))
x = s_eigh[1].dot(s_diag).dot(s_eigh[1].T)
x.T.dot(s).dot(x)

array([[ 1.00000000e+00, -3.24696867e-15,  3.81639165e-17,
         3.89958912e-17,  2.68882139e-17, -6.73072709e-16,
         7.80625564e-16,  6.93889390e-18,  9.42819196e-18,
        -2.08166817e-16, -1.45716772e-16,  7.38992201e-16,
         3.85108612e-16,  1.08676081e-17,  1.38777878e-16],
       [-2.52662474e-15,  1.00000000e+00, -3.92914867e-16,
        -1.18507569e-16, -2.02312125e-16,  2.26728358e-15,
         4.31946146e-16, -1.11369247e-15,  1.33932776e-17,
        -1.49186219e-16,  1.61069075e-15, -1.68268177e-16,
        -1.05427819e-15,  1.39485233e-17,  4.02455846e-16],
       [ 1.83880688e-16, -5.75060832e-16,  1.00000000e+00,
         1.56088760e-16, -2.22044605e-16,  2.15105711e-16,
        -3.19189120e-16, -2.94035629e-16, -1.13677680e-17,
         4.16333634e-17,  3.46944695e-17, -2.22044605e-16,
        -1.21430643e-17, -1.04827802e-17,  2.77555756e-16],
       [ 3.89958912e-17, -1.18507569e-16,  1.56088760e-16,
         1.00000000e+00,  1.32922707e-14,  6.21299434

In [75]:
a = h_test.id.getDensityMatrix('alpha')
b = h_test.id.getDensityMatrix('beta')
x_i = np.linalg.inv(x)
x_i_t = np.linalg.inv(x.T)
a_trans = x_i.dot(a).dot(x_i_t)
b_trans = x_i.dot(b).dot(x_i_t)
p = (a_trans + b_trans)/2
c = eigh(p)
c

(array([-6.00687650e-18, -4.75866979e-18, -3.31473633e-18, -2.27268212e-18,
        -2.20868761e-18, -7.67134458e-33,  4.58363817e-34,  1.80030792e-33,
         5.22874328e-19,  1.93073916e-17,  4.00913763e-17,  6.23349833e-17,
         4.44089210e-16,  5.00000000e-01,  1.00000000e+00]),
 array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          6.16588591e-01,  6.31105095e-01, -4.70664284e-01],
        [ 1.28778561e-01, -1.33799819e-02, -9.08902394e-02,
          2.20571569e-01, -4.32896469e-02,  3.40471047e-15,
         -1.87520514e-16, -1.51128824e-15, -1.38953325e-01,
         -1.03370758e-01,  2.25640737e-01,  6.14706453e-01,
         -5.36766113e-01,  3.19698738e-01, -2.74506401e-01],
        [-1.09393643e-01,  3.48007069e-01, -2.88241726e-02,
          2.71966661e-02,  3.98216954e-01, -1.693

Up untill here everything should be correct (density matrices are idempotent, p matrix has correct eigenvalues). The area below is still under construction.

In [90]:
f_a = h_test.id.displayFockMatrix('alpha')
f_b = h_test.id.displayFockMatrix('beta')

In [95]:
f_cs = (f_a + f_b)/2

In [96]:
f_trans = c[1].T.dot(f_a).dot(c[1])
g_trans = c[1].T.dot(f_b).dot(c[1])
closed_shell_trans = c[1].T.dot(f_cs).dot(c[1])

In [97]:
f_trans[:3, 3:] = closed_shell_trans[:3, 3:]
        
f_trans[3:, :3] = closed_shell_trans[3:, :3]

In [98]:
g_trans[:3, 3:] = closed_shell_trans[:3, 3:]
        
g_trans[3:, :3] = closed_shell_trans[3:, :3]

In [99]:
h_test.id.setGuess(f_trans, "alpha")
h_test.id.setGuess(g_trans, "beta")

h_test.id.iterator()

iteration: 0, E_tot:  4.73457110, E_elek:  3.14703947, deltaE:  3.14703947, rmsD:  43.60713786
iteration: 1, E_tot: -1.43084945, E_elek: -3.01838108, deltaE: -6.16542055, rmsD:  3.13559010
iteration: 2, E_tot: -1.49995285, E_elek: -3.08748448, deltaE: -0.06910340, rmsD:  0.81419824
iteration: 3, E_tot: -1.50530721, E_elek: -3.09283885, deltaE: -0.00535437, rmsD:  0.26755229
iteration: 4, E_tot: -1.50596161, E_elek: -3.09349325, deltaE: -0.00065440, rmsD:  0.11889826
iteration: 5, E_tot: -1.50609427, E_elek: -3.09362590, deltaE: -0.00013265, rmsD:  0.08024243
iteration: 6, E_tot: -1.50613710, E_elek: -3.09366873, deltaE: -0.00004283, rmsD:  0.06740736
iteration: 7, E_tot: -1.50615669, E_elek: -3.09368832, deltaE: -0.00001959, rmsD:  0.05947435
iteration: 8, E_tot: -1.50616818, E_elek: -3.09369981, deltaE: -0.00001149, rmsD:  0.05452963
iteration: 9, E_tot: -1.50617631, E_elek: -3.09370794, deltaE: -0.00000812, rmsD:  0.05117593
iteration: 10, E_tot: -1.50618284, E_elek: -3.09371448, del

iteration: 96, E_tot: -1.50627394, E_elek: -3.09380557, deltaE: -0.00000003, rmsD:  0.00295554
iteration: 97, E_tot: -1.50627396, E_elek: -3.09380559, deltaE: -0.00000002, rmsD:  0.00286131
iteration: 98, E_tot: -1.50627398, E_elek: -3.09380562, deltaE: -0.00000002, rmsD:  0.00277009
iteration: 99, E_tot: -1.50627401, E_elek: -3.09380564, deltaE: -0.00000002, rmsD:  0.00268179
iteration: 100, E_tot: -1.50627403, E_elek: -3.09380566, deltaE: -0.00000002, rmsD:  0.00259632
iteration: 101, E_tot: -1.50627404, E_elek: -3.09380568, deltaE: -0.00000002, rmsD:  0.00251357
iteration: 102, E_tot: -1.50627406, E_elek: -3.09380569, deltaE: -0.00000002, rmsD:  0.00243347
iteration: 103, E_tot: -1.50627408, E_elek: -3.09380571, deltaE: -0.00000002, rmsD:  0.00235592
iteration: 104, E_tot: -1.50627409, E_elek: -3.09380572, deltaE: -0.00000002, rmsD:  0.00228085
iteration: 105, E_tot: -1.50627411, E_elek: -3.09380574, deltaE: -0.00000001, rmsD:  0.00220819
iteration: 106, E_tot: -1.50627412, E_elek: 

iteration: 192, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00013272
iteration: 193, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00012850
iteration: 194, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00012442
iteration: 195, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00012047
iteration: 196, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00011664
iteration: 197, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00011293
iteration: 198, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00010934
iteration: 199, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00010587
iteration: 200, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00010251
iteration: 201, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00009925
iteration: 202, E_tot: -1.50627432, E_el

iteration: 288, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00000598
iteration: 289, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00000579
iteration: 290, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00000561
iteration: 291, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00000543
iteration: 292, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00000526
iteration: 293, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00000509
iteration: 294, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00000493
iteration: 295, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00000477
iteration: 296, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00000462
iteration: 297, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00000447
iteration: 298, E_tot: -1.50627432, E_el

-1.5062743202993434

array([[ 9.81606608e-03,  2.12337150e-03, -9.32538710e-03,
         1.23478390e-02,  1.15462845e-02,  2.43780973e-17,
         2.07033311e-19, -1.68541487e-16,  3.97970059e-03,
         6.80124992e-03,  2.52635117e-02,  4.28434112e-02,
         1.93208552e-02,  1.13075201e-01,  1.06816670e-02],
       [ 2.12337150e-03,  4.85293995e-03, -4.21998308e-03,
         6.89991101e-03,  9.11731437e-03, -1.64002573e-16,
         1.86410313e-17, -1.33686189e-16, -2.98370923e-03,
        -1.79182976e-03,  2.65430680e-03,  1.30334059e-03,
        -7.63839248e-03,  4.01027056e-03,  2.87471554e-02],
       [-9.32538710e-03, -4.21998308e-03,  9.96359194e-03,
        -1.38507618e-02, -1.42879048e-02,  6.17074342e-17,
        -9.51996098e-18,  2.08862048e-16, -1.85327482e-03,
        -4.82533570e-03, -2.25915578e-02, -3.67088246e-02,
        -1.24301716e-02, -9.71703645e-02, -2.34017474e-02],
       [ 1.23478390e-02,  6.89991101e-03, -1.38507618e-02,
         1.96029200e-02,  2.08957742e-02, -1.32263007

In [97]:
a = h_test.id.getDensityMatrix("alpha")
a_trans = z.T.dot(a).dot(z)

a_trans

array([[ 7.86993446e-23, -4.40492022e-29, -1.60209546e-24,
         1.10167228e-16,  6.31851038e-13,  6.80554947e-13,
        -2.47474297e-16, -6.62364160e-13,  5.36309038e-14,
         2.54238109e-14,  3.77365032e-13,  7.03392293e-13,
        -5.30833893e-14,  4.65973030e-12, -8.89193826e-12],
       [-4.40492022e-29,  2.76036507e-35,  8.96723218e-31,
        -6.16626961e-23, -2.50296198e-19, -5.15076464e-19,
         1.91198853e-22,  4.01685377e-19, -6.81987441e-20,
        -1.00010858e-20, -2.88968418e-19, -5.07425163e-19,
         2.10958303e-20, -1.66079758e-18,  5.69177369e-18],
       [-1.60209546e-24,  8.96723218e-31,  3.26141200e-26,
        -2.24269232e-18, -1.28624713e-14, -1.38544592e-14,
         5.03798893e-18,  1.34839228e-14, -1.09185579e-15,
        -5.17547528e-16, -7.68225020e-15, -1.43193185e-14,
         1.08060860e-15, -9.48568423e-14,  1.81016193e-13],
       [ 1.10167228e-16, -6.16626961e-23, -2.24269232e-18,
         1.54217524e-10,  8.84480564e-07,  9.52694805

In [45]:
a_trans.dot(a_trans)

array([[ 1.31509744e-32, -1.14508632e-33,  3.39607801e-48,
        -1.61963005e-49,  2.05562799e-49, -9.73656549e-38,
         6.24182271e-35, -1.09179883e-33, -6.00573899e-34,
         8.81299316e-33, -6.04751451e-33, -1.33620208e-33,
         6.66989478e-33,  5.38281907e-17, -3.80353931e-17],
       [ 2.55329854e-33,  1.82436891e-33,  1.34535402e-47,
         1.56847134e-49,  1.23531439e-48,  4.82548374e-37,
         1.88662530e-34, -4.78299584e-34,  1.31304341e-33,
         5.06955630e-34, -8.48027587e-33,  3.23997069e-34,
         1.42303235e-33,  5.40898234e-18,  5.72211595e-17],
       [ 1.46042575e-47,  1.16065337e-47,  8.83747223e-62,
         1.07094341e-63,  8.17295333e-63,  3.26345890e-51,
         1.35252332e-48, -2.70869588e-48,  9.32448840e-48,
         2.66324006e-48, -5.41008378e-47,  2.35858984e-48,
         8.41908870e-48,  3.15960266e-32,  3.71914730e-31],
       [ 1.79514095e-50,  1.35123144e-49,  8.48917413e-64,
         1.24083986e-65,  7.91306457e-65,  3.28005275

In [32]:
c = eigh(a)[0]

In [33]:
b = h_test.id.getDensityMatrix("beta")
b.trace()

0.4718682903812628

In [34]:
d = eigh(b)[0]

In [35]:
e = np.diag(c) + np.diag(d)
e/2

array([[-1.01658530e-17,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00, -3.29458077e-18,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00, -2.57922657e-19,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        -2.53354521e-19,  0.00000000e+00,  0.00000000